In [60]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")

snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

In [50]:
sectors = snp500['sector'].unique().tolist()
sector_constant_list = [('PROFILE_SECTOR_' + sector_key.upper().replace(" ", "_") + '=\'' + sector_key + '\'')
                        for
                        sector_key in
                        sectors]

with open("sector_constant.txt", "a") as f:
    print(*sector_constant_list, sep="\n", file=f)

PROFILE_SECTOR_INDUSTRIALS='Industrials'
PROFILE_SECTOR_HEALTH_CARE='Health Care'
PROFILE_SECTOR_INFORMATION_TECHNOLOGY='Information Technology'
PROFILE_SECTOR_COMMUNICATION_SERVICES='Communication Services'
PROFILE_SECTOR_CONSUMER_STAPLES='Consumer Staples'
PROFILE_SECTOR_CONSUMER_DISCRETIONARY='Consumer Discretionary'
PROFILE_SECTOR_UTILITIES='Utilities'
PROFILE_SECTOR_FINANCIALS='Financials'
PROFILE_SECTOR_MATERIALS='Materials'
PROFILE_SECTOR_REAL_ESTATE='Real Estate'
PROFILE_SECTOR_ENERGY='Energy'


In [65]:
sub_sectors = snp500['subSector'].unique().tolist()
sector_constant_list = [('PROFILE_INDUSTRY_' +
                         sector_key.upper().
                         replace("&", "").
                         replace("-", "_").replace(", ", "").replace(" ", "_").replace("__",
                                                                                       "_") + '=\'' + sector_key + '\'')
                        for
                        sector_key in
                        sub_sectors]
with open("sub_sector_constant.txt", "w") as f:
    print(*sector_constant_list, sep="\n", file=f)

In [15]:
title = 'company-profile'
from datetime import date

today = date.today()

file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'

df_snp500 = snp500.copy()
df_snp500.set_index('symbol', inplace=True)
df_snp500.to_csv(file_name, index=True)

In [2]:
title = 'financial-ratios'
period = 'quarter'

from tqdm import tqdm
import pandas as pd
from datetime import date

symbols = snp500['symbol']
fmp_financial_ratios = []
for symbol in tqdm(symbols):
    fmp_financial_ratios += fmpsdk.financial_ratios(apikey=apikey, symbol=symbol, limit=-1, period=period)

df_financial_ratios = pd.DataFrame(fmp_financial_ratios, index=None)

today = date.today()
fmp_api_name = title + '-' + period
file_name = fmp_api_name + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_financial_ratios.to_csv(file_name, index=False)

100%|██████████| 503/503 [11:09<00:00,  1.33s/it]


In [57]:
title = 'economic-indicators'

from datetime import date

today = date.today()
from urllib.request import urlopen
import certifi
import json
import os
from dotenv import load_dotenv

from_date = '1962-01-01'
to_date = today.strftime("%Y-%m-%d")
economic_indicators = ['CPI', 'inflationRate', 'inflation', 'unemploymentRate']

load_dotenv()
apikey = os.environ.get("apikey")


def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)


def economic_indicator(topic: str):
    url = "https://financialmodelingprep.com/api/v4/economic?name={}&from={}&to={}&apikey={}".format(
        topic, from_date, to_date, apikey)
    fmp_economic_indicator = get_jsonparsed_data(url)
    return [{'date': item['date'], topic: item['value']} for item in fmp_economic_indicator]


result = []
for topic in economic_indicators:
    result += (economic_indicator(topic=topic))

df_economic_indicators = pd.DataFrame(result)
df_economic_indicators.set_index('date', inplace=True)
df_economic_indicators.index = pd.to_datetime(df_economic_indicators.index)
df_economic_indicators = df_economic_indicators.sort_index()

import numpy as np

df_economic_indicators = df_economic_indicators.groupby(df_economic_indicators.index).sum()
df_economic_indicators.replace(0, np.nan, inplace=True)
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_economic_indicators.to_csv(file_name, index=True)

C:\Users\JBS\AppData\Local\Temp\ipykernel_19092\666637363.py:21: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [17]:
title = 'market-index'
from datetime import date

today = date.today()
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")

from_date = '1962-01-01'
to_date = today.strftime("%Y-%m-%d")
index_list = pd.DataFrame(fmpsdk.available_indexes(apikey=apikey))
target_index_list = ['^GSPC', '^TYX', '^FVX', '^TNX']

target_index_history = []
for target_index in target_index_list:
    for item in fmpsdk.historical_price_full(apikey=apikey, symbol=target_index, from_date=from_date, to_date=to_date):
        target_index_history.append({'symbol': target_index, 'date': item['date'], 'price': item['adjClose']})

df_market_index = pd.DataFrame(target_index_history)
df_market_index = df_market_index.pivot(index='date', columns='symbol', values='price')
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_index.to_csv(file_name, index=True)

In [23]:
title = 'market-capitalization'
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()

apikey = os.environ.get("apikey")

result = []
for symbol in tqdm(snp500['symbol']):
    result += fmpsdk.historical_market_capitalization(symbol=symbol, apikey=apikey, limit=-1)

from datetime import date

today = date.today()
df_market_capitalization = pd.DataFrame(result)
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_capitalization.to_csv(file_name, index=False)

100%|██████████| 503/503 [12:38<00:00,  1.51s/it]


In [9]:
import pandas as pd
from datetime import date
title = 'market-capitalization'

today = date.today()
df_market_capitalization = pd.read_csv('market-capitalization-20220828.csv')[::4]
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_capitalization.to_csv(file_name, index=False)

In [45]:
title = 'etc-history'
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()
apikey = os.environ.get("apikey")

df_bitcoin_history = \
    pd.DataFrame(fmpsdk.historical_price_full(symbol='BTCUSD', apikey=apikey)).set_index('date').sort_index()['adjClose']
df_bitcoin_history.rename('BTCUSD', inplace=True)

df_covid_history = pd.read_csv('covid-case.csv', index_col='date')['cases']
df_covid_history.rename('covidCases', inplace=True)
df_etc = pd.concat([df_covid_history, df_bitcoin_history], axis=1, ).sort_index()

from datetime import date

today = date.today()
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_etc.to_csv(file_name, index=True)

In [70]:
title = 'financial-statements'

from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()
apikey = os.environ.get("apikey")

financial_statements = dict()
for period in ['annual', 'quarter']:
    fmp_income_statement = []
    fmp_balance_sheet = []
    fmp_cashflow_statement = []
    for symbol in tqdm(snp500['symbol']):
        fmp_income_statement += fmpsdk.income_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
        fmp_balance_sheet += fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
        fmp_cashflow_statement += fmpsdk.cash_flow_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
    financial_statements.update({period: [fmp_income_statement, fmp_balance_sheet, fmp_cashflow_statement]})

100%|██████████| 503/503 [25:41<00:00,  3.06s/it]


In [77]:
from datetime import date

today = date.today()
for period in ['annual', 'quarter']:
    file_name = title + '-' + period + '-' + (today.strftime("%Y%m%d")) + '.csv'
    df_financial_statements = pd.concat([pd.DataFrame(item) for item in financial_statements[period]], axis=1)
    df_financial_statements.to_csv(file_name, index=False)


In [81]:
columns = df_financial_statements.columns
with open("financial_statements_list.txt", "a") as f:
    print(*columns, sep="\n", file=f)

In [ ]:
# TODO

import pickle
import pandas as pd

with open('snp500_price_list_19620102_20220814.pickle', 'rb') as file:
    data = pickle.load(file)

df_data = pd.DataFrame(data)
df_data.to_csv('snp500_price_list_19620102_20220814.csv', index=True, index_label='date')


In [16]:
import pandas as pd

df_result = pd.DataFrame(result)